<a href="https://colab.research.google.com/github/Manjog/Text-Summarizer-project/blob/main/2ndimp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
## for data
!pip install datasets
!pip install contractions
!pip install rouge
import datasets  #(1.13.3)
import pandas as pd  #(0.25.1)
import numpy  #(1.16.4)
## for plotting
import matplotlib.pyplot as plt  #(3.1.2)
import seaborn as sns  #(0.9.0)
## for preprocessing
import re
import nltk  #(3.4.5)
import contractions  #(0.0.18)
## for evaluation
import rouge  #(1.0.0)
import difflib
## for seq2seq
from tensorflow.keras import callbacks, models, layers, preprocessing as kprocessing #(2.6.0)
!pip install scikit-learn

In [2]:
## load the full dataset of 300k articles
dataset = datasets.load_dataset("cnn_dailymail", '3.0.0')
lst_dics = [dic for dic in dataset["train"]]
## keep the first N articles if you want to keep it lite
dtf = pd.DataFrame(lst_dics).rename(columns={"article":"text",
      "highlights":"y"})[["text","y"]].head(20000)
dtf.head()

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

,text,y
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."


In [4]:
i = 1
print("--- Full text ---")
print(dtf["text"][i])
print("--- Summary ---")
print(dtf["y"][i])

--- Full text ---
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less li

In [12]:

from sklearn.model_selection import train_test_split
#from sklearn.models_selection import split_train_test
X_train, X_test, y_train, y_test= train_test_split(dtf['text'], dtf['y'], test_size=0.2, random_state=10)

In [18]:
dtf_train = dtf.iloc[i+1:]
dtf_test = dtf.iloc[:i+1]
print(dtf_test)

                                                text  \
0  LONDON, England (Reuters) -- Harry Potter star...   
1  Editor's note: In our Behind the Scenes series...   

                                                   y  
0  Harry Potter star Daniel Radcliffe gets £20M f...  
1  Mentally ill inmates in Miami are housed on th...  


In [19]:
dtf_train.shape

(19998, 2)

In [20]:
dtf_test.shape

(2, 2)

In [ ]:
dtf_test.shape

(2, 2)

In [32]:
def textrank(corpus, ratio=0.2):
 if type(corpus) is str:
   corpus = [corpus]
 lst_summaries = [gensim.summarization.summarize(txt,ratio=ratio) for txt in corpus]
 return lst_summaries
 ## Apply the function to corpus
 predicted =textrank(corpus=dtf_test["text"], ratio=0.2)
 predicted[i]
 print(predicted[i])

In [35]:
def evaluate_summary(y_test, predicted):
 rouge_score =rouge.Rouge()
 scores = rouge_score.get_scores(y_test, predicted, avg=True)
 score_1 = round(scores['rouge-1']['f'], 2)
 score_2 = round(scores['rouge-2']['f'], 2)
 score_L = round(scores['rouge-l']['f'], 2)
 print("rouge1:", score_1, "| rouge2:", score_2, "| rougeL:",
 score_2, "--> avg rouge:", round(np.mean([score_1,score_2,score_L]), 2))
 ## Apply the function to predicted
 i = 5
 evaluate_summary(dtf_test["y"][i], predicted[i])

In [53]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
## create stopwords
lst_stopwords =nltk.corpus.stopwords.words("english")
## add words that are too frequent
lst_stopwords = lst_stopwords + ["cnn","say","said","new"]
## cleaning function
def utils_preprocess_text(txt, punkt=True, lower=True, slang=True, lst_stopwords=None, stemm=False, lemm=True):
### separate sentences with '. '
 txt = re.sub(r'\.(?=[^ \W\d])', '. ', str(txt))
### remove punctuations and characters
 txt = re.sub(r'[^\w\s]', '', txt) if punkt is True else txt
### strip
 txt = " ".join([word.strip() for word in txt.split()])
### lowercase
 txt = txt.lower() if lower is True else txt
### slang
 txt = contractions.fix(txt) if slang is True else txt
### tokenize (convert from string to list)
 lst_txt = txt.split()
### stemming (remove -ing, -ly, ...)
 if stemm is True:
   ps = nltk.stem.porter.PorterStemmer()
   lst_txt = [ps.stem(word) for word in lst_txt]
### lemmatization (convert the word into root word)
 if lemm is True:
  lem = nltk.stem.wordnet.WordNetLemmatizer()
  lst_txt = [lem.lemmatize(word) for word in lst_txt]
### remove Stopwords
  if lst_stopwords is not None:
   lst_txt = [word for word in lst_txt if word not in lst_stopwords]
### back to string
  txt = " ".join(lst_txt)
  return txt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [46]:
dtf['text'] [1]

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [54]:
dtf_train["text_clean"] = dtf_train["text"].apply(lambda x: utils_preprocess_text(x, punkt=True, lower=True, slang=True, lst_stopwords=lst_stopwords, stemm=False, lemm=True))


<ipython-input-54-23e5ff00cbd3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtf_train["text_clean"] = dtf_train["text"].apply(lambda x: utils_preprocess_text(x, punkt=True, lower=True, slang=True, lst_stopwords=lst_stopwords, stemm=False, lemm=True))


In [57]:
print(dtf_train["text_clean"])

2        minneapolis minnesota driver minneapolis bridg...
3        washington doctor removed five small polyp pre...
4        national football league ha indefinitely suspe...
5        baghdad iraq dressed superman shirt 5yearold y...
6        baghdad iraq woman afraid ashamed show face re...
                               ...                        
19995    los angeles 24yearold connecticut man affiliat...
19996    jerusalem israeli official working repair ceme...
19997    york city quintessential metropolis city city ...
19998    los angeles film festival june 18 28 since osc...
19999    ten year ago nbc eager come version unexpected...
Name: text_clean, Length: 19998, dtype: object


In [58]:
dtf_train["y_clean"] = dtf_train["y"].apply(lambda x: utils_preprocess_text(x, punkt=True, lower=True, slang=True, lst_stopwords=lst_stopwords, stemm=False, lemm=True))


<ipython-input-58-c4de8c2d38a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtf_train["y_clean"] = dtf_train["y"].apply(lambda x: utils_preprocess_text(x, punkt=True, lower=True, slang=True, lst_stopwords=lst_stopwords, stemm=False, lemm=True))


In [59]:
print(dtf_train["y_clean"])

2        thought wa going die driver man pickup truck w...
3        five small polyp found procedure none worrisom...
4        nfl chief atlanta falcon owner critical michae...
5        parent beam pride cannot stop smiling outpouri...
6        aid worker violence increased cost living driv...
                               ...                        
19995    kevin george poe 24 connecticut accused anonym...
19996    israeli president crime terrible everything st...
19997    york city frenetic glamorous quintessential me...
19998    l film festival offer chance enjoy cinema spir...
19999    david bianculli nbcs fear factor redux even di...
Name: y_clean, Length: 19998, dtype: object
